In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import make_classification
from sklearn import metrics


df = pd.read_csv('parlers-data2/part-00000-4cd6e305-e7f2-4779-bc4b-8ef43772fdd0-c000 copy.csv')

train, val = train_test_split(df, test_size=0.3)
train, test = train_test_split(train, test_size=0.1)
print('%.2f %%' % float(len(train)/len(df)*100))
print('%.2f %%' % float(len(val)/len(df)*100))
print('%.2f %%' % float(len(test)/len(df)*100))

#Split the data into x and y
train_x = train[['Comments', 'Followers', 'Following', 'Reposts', 'verified']]
train_y = train['upvotes']
val_x = train[['Comments', 'Followers', 'Following', 'Reposts', 'verified']]
val_y = train['upvotes']
test_x = train[['Comments', 'Followers', 'Following', 'Reposts', 'verified']]
test_y = train['upvotes']

61.02 %
30.51 %
8.47 %


In [2]:
train_x.head()

,Comments,Followers,Following,Reposts,verified
48,29,20000,25000,129,False
56,23,15000,74,26,False
0,2,12000,2900,16,False
29,29,20000,25000,129,False
46,29,20000,25000,129,False


In [12]:
#Parameters to consider: n_estimators, criterion, max_depth, min_samples_split
def random_forest_model(train_x, train_y, val_x, val_y):
    rf = RandomForestRegressor()
    rf.fit(train_x, train_y)
    val_y_pred = rf.predict(val_x)
    val_y_pred = pd.DataFrame(val_y_pred, columns=['val_predicted_upvotes'])
    val_comparison = val_y_pred.join(val_y)
    return rf, val_y_pred, val_comparison

In [16]:
def random_forest_test(model, test_x, test_y):
    # Make predictions using the test set
    test_y_pred = model.predict(test_x)
    test_y_pred = pd.DataFrame(test_y_pred, columns=['test_predicted_upvotes'])
    test_comparison = test_y_pred.join(test_y)
    
    return test_y_pred, test_comparison

In [31]:
def random_forest_metrics(y_test, y_pred, comparison):
    groundtruth = comparison.to_numpy()
    acc = [i for i in groundtruth if i[0] == i[1]]
    acc = len(acc)

    print('Accuracy: %.2f %%' % float(acc/len(y_test)*100))
    
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))


In [33]:
val_comparison.head()

,val_predicted_upvotes,upvotes
0,120.0,NaN
1,120.0,NaN
2,120.0,69.0
3,120.0,NaN
4,120.0,NaN


In [32]:
model, val_y_pred, val_comparison = random_forest_model(train_x, train_y, val_x, val_y)
print('VALIDATION METRICS')
random_forest_metrics(val_y, val_y_pred, val_comparison)
print()
test_y_pred, test_comparison = random_forest_test(model, test_x, test_y)
print('TEST METRICS')
random_forest_metrics(test_y, test_y_pred, test_comparison)

VALIDATION METRICS
Accuracy: 47.22 %
Mean Absolute Error: 1.0486111111111112
Mean Squared Error: 8.595569444444443
Root Mean Squared Error: 2.931820158953213

TEST METRICS
Accuracy: 47.22 %
Mean Absolute Error: 1.0486111111111112
Mean Squared Error: 8.595569444444443
Root Mean Squared Error: 2.931820158953213
